In [2]:
from datasets import load_from_disk

In [14]:
d = load_from_disk("/fsx/phuc/projects/nanotron/examples/infinite-context-length/data/eval/1024/needle_eval_data_and_1024_ctx_and_depth_50_and_id_42_and_num_shots_2")

In [16]:
d[0]["prompt"]

'There is a pass key hidden inside a lot of irrelevant text. Find it and memorize them. I will quiz you about what is the pass key later on. \n\n Passage: The gatecode is 312. Remember it. 312 is the gatecode. The grass is green. The sky is blue. The sun is yellow. Here we go. There and back again. The grass is green. The sky is blue. The sun is yellow. Here we go. There and back again. What is the gatecode? The gatecode is 312. \n\n Passage: The grass is green. The sky is blue. The sun is yellow. Here we go. There and back again. The vault key is 415. Remember it. 415 is the vault key. The grass is green. The sky is blue. The sun is yellow. Here we go. There and back again. What is the gatecode? The vault key is 415. \n \n Passage:  The grass is green. The sky is blue. The sun is yellow. Here we go. There and back again. The grass is green. The sky is blue. The sun is yellow. Here we go. There and back again. The grass is green. The sky is blue. The sun is yellow. Here we go. There an

In [1]:
def load_and_filter_dataset(eval_dataset_path, depth_percent, num_shots, num_digits, seed, num_samples):
    import random
    from datasets import load_dataset
    # Set seeds for reproducibility
    random.seed(seed)
    
    # Load the dataset
    dataset = load_dataset(eval_dataset_path, split="train")
    
    # Filter the dataset
    filtered_dataset = dataset.filter(
        lambda x: x["depth_percent"] == depth_percent and 
                x["num_shots"] == num_shots
    )
    if num_digits > 0:
        filtered_dataset = filtered_dataset.filter(
            lambda x: x["num_digits"] == num_digits
        )

    # filtered_dataset = dataset.filter(
    #     lambda x: x["depth_percent"] == depth_percent and 
    #             x["num_shots"] == num_shots and 
    #             x["num_digits"] == num_digits
    # )
    
    # Shuffle the dataset deterministically
    shuffled_dataset = filtered_dataset.shuffle(seed=seed)
    
    # Select only the first 10 samples
    final_dataset = shuffled_dataset.select(range(min(num_samples, len(shuffled_dataset))))
    
    return final_dataset

In [3]:
samples = load_and_filter_dataset("nanotron/llama3-1024-passkey-retrieval-eval", depth_percent=0, num_shots=0, num_digits=2, seed=42, num_samples=10)

In [5]:
samples.to_pandas()

,id,prompt,answer,context_length,token_count,num_shots,num_digits,depth_percent
0,2d454feb-59a8-4f39-8ad3-c9ed589e5640,There is a pass key hidden inside a lot of irr...,80,1024,1004,0,2,0
1,6712605d-21d8-4b1e-8248-fc6a321fd59e,There is a pass key hidden inside a lot of irr...,87,1024,1004,0,2,0
2,0b8734b1-a140-4cf0-a461-ed686fed566a,There is a pass key hidden inside a lot of irr...,85,1024,1004,0,2,0
3,b9035923-f43d-487c-aef1-5027a10ef3ae,There is a pass key hidden inside a lot of irr...,11,1024,1004,0,2,0
4,5c917b9f-dae3-4e07-a733-6ac089a4e75c,There is a pass key hidden inside a lot of irr...,14,1024,1004,0,2,0
5,86e5fd36-951e-4333-9306-2c8045ced06d,There is a pass key hidden inside a lot of irr...,54,1024,1004,0,2,0
6,6302d923-e9e4-4319-99f5-14daa43408ed,There is a pass key hidden inside a lot of irr...,83,1024,1004,0,2,0
7,8c90e431-bce5-4792-90c2-a765e94ceb91,There is a pass key hidden inside a lot of irr...,92,1024,1004,0,2,0
8,b54dc476-b4e7-48b3-b639-599527606858,There is a pass key hidden inside a lot of irr...,13,1024,1004,0,2,0
9,48c4b32e-911b-45e6-a53e-c721b187df8d,There is a pass key hidden inside a lot of irr...,93,1024,1004,0,2,0


In [7]:
samples[0]["prompt"]

'There is a pass key hidden inside a lot of irrelevant text. Find it and memorize them. I will quiz you about what is the pass key later on. \n . The pass key is 80. Remember it. 80 is the pass key. \n Passage:  The grass is green. The sky is blue. The sun is yellow. Here we go. There and back again. The grass is green. The sky is blue. The sun is yellow. Here we go. There and back again. The grass is green. The sky is blue. The sun is yellow. Here we go. There and back again. The grass is green. The sky is blue. The sun is yellow. Here we go. There and back again. The grass is green. The sky is blue. The sun is yellow. Here we go. There and back again. The grass is green. The sky is blue. The sun is yellow. Here we go. There and back again. The grass is green. The sky is blue. The sun is yellow. Here we go. There and back again. The grass is green. The sky is blue. The sun is yellow. Here we go. There and back again. The grass is green. The sky is blue. The sun is yellow. Here we go. 

In [19]:
from datasets import load_dataset

In [20]:
ds = load_dataset("nanotron/llama3-1024-passkey-retrieval-eval")

In [21]:
def remove_end_number(text):
    # Extract the last 30 characters
    last_30 = text[-30:]
    
    # Find the last occurrence of "is "
    last_is_index = last_30.rfind("is ")
    
    if last_is_index != -1:
        # Remove everything after "is "
        result = text[:len(text) - (30 - last_is_index - 3)]
        return result
    else:
        # If "is " is not found in the last 30 characters, return the original string
        return text

In [22]:
def process_example(example):
    example['prompt'] = remove_end_number(example['prompt'])
    return example

# Apply the function to each example in the dataset
processed_dataset = ds['train'].map(process_example)

# Print a few examples to verify
# for i in range(5):
#     print(f"Original: {processed_dataset[i]['prompt']}")
#     print(f"Processed: {processed_dataset[i]['processed_text']}")
#     print()

# Save the processed dataset if needed
# processed_dataset.save_to_disk('path/to/save/processed_dataset')

Map: 100%|█████████████████████████████████████████████████████████████████████████████████| 12600/12600 [00:00<00:00, 13010.74 examples/s]


In [25]:
processed_dataset[3]["prompt"]

'There is a pass key hidden inside a lot of irrelevant text. Find it and memorize them. I will quiz you about what is the pass key later on. \n\n Passage: The gatecode is 312. Remember it. 312 is the gatecode. The grass is green. The sky is blue. The sun is yellow. Here we go. There and back again. The grass is green. The sky is blue. The sun is yellow. Here we go. There and back again. What is the gatecode? The gatecode is 312. \n\n Passage: The grass is green. The sky is blue. The sun is yellow. Here we go. There and back again. The vault key is 415. Remember it. 415 is the vault key. The grass is green. The sky is blue. The sun is yellow. Here we go. There and back again. What is the gatecode? The vault key is 415. \n \n Passage:  The grass is green. The sky is blue. The sun is yellow. Here we go. There and back again. The grass is green. The sky is blue. The sun is yellow. Here we go. There and back again. The grass is green. The sky is blue. The sun is yellow. Here we go. There an

In [26]:
processed_dataset.push_to_hub(
    repo_id="nanotron/llama3-1024-passkey-retrieval-eval",
    # repo_type="dataset",
    # private=True,  # Set to False if you want the dataset to be public
    # token="your_access_token"
)

Uploading the dataset shards: 100%|██████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.08it/s]


CommitInfo(commit_url='https://huggingface.co/datasets/nanotron/llama3-1024-passkey-retrieval-eval/commit/8f3b79e9ffb94efbc49d73978ddbc705d6f14bc9', commit_message='Upload dataset', commit_description='', oid='8f3b79e9ffb94efbc49d73978ddbc705d6f14bc9', pr_url=None, pr_revision=None, pr_num=None)